Data sources: [The Japanese Mortality Database](https://www.ipss.go.jp/p-toukei/JMD/index-en.asp) → "Life Tables"<br>
wiki > [List of Japanese prefectures by life expectancy](https://en.wikipedia.org/wiki/List_of_Japanese_prefectures_by_life_expectancy) <i>[(alternative)](https://en.wikipedia.org/wiki/User:Lady3mlnm/List_of_Japanese_prefectures_by_life_expectancy_(alternative))</i> / [Продолжительность жизни в префектурах Японии](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_префектурах_Японии)<br />
[selection of colors](http://mal-bioit.ru/survey-web-colors)

Data files from [All Japan: Life Table Data Series](https://www.ipss.go.jp/p-toukei/JMD/00/index-en.html) have to be prepared preliminary with taking replacement " +" to "\t" in a text editor like Notepad++.

In [3]:
import pandas as pd
import re
import math

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [4]:
# pd.options.display.min_rows=4

In [5]:
YEAR = 2023
SORTED_BY = ['t_0', 't_65']

In [6]:
def read_single_csv(file_name, year):
    df_c = pd.read_csv(f"data-Life_tables/{file_name}", sep='\t', usecols=['Year', 'Age', 'ex'], skiprows=2)
    df_c = df_c.loc[df_c['Year'] == year] \
               .drop(columns='Year') \
               .set_index('Age')
    df_c = df_c.rename(index={'110+': '110'}).rename(index=lambda st: int(st))
    df_c = df_c.loc[[0, 65, 80, 100, 110]]
    return df_c


pattern_region_general = re.compile(r'(?<=^[\d.]{3}).*(?=(.ken|.to|.fu), Life tables)')  # what is in the most cases, like '47.Okinawa.ken'
pattern_region_1       = re.compile(r'(?<=^[\d.]{3}).*(?=, Life tables)')  # case of '01.Hokkaido' in the file Jp_01.csv
pattern_region_0       = re.compile(r'(?<=^).*(?=, Life tables)')  # case of 'Japan' in the file Jp_01.csv

def extract_name_of_region_from_file(region_nmb):
    with open(f"data-Life_tables/JpLT_{region_nmb:02}-total.csv", 'r') as fh:
        title_line = fh.readline()
        if region_nmb >= 2:
            fragment_name_found = re.search(pattern_region_general, title_line)
        elif region_nmb == 1:
            fragment_name_found = re.search(pattern_region_1, title_line)
        elif region_nmb == 0:
            fragment_name_found = re.search(pattern_region_0, title_line)
        else:
            raise ValueError(f"Not expected region number: {region_nmb}")
        name = fragment_name_found.group() if fragment_name_found else 'region name is not found'
    return name



def read_single_region(region_nmb):
    df_t = read_single_csv(f"JpLT_{region_nmb:02}-total.csv", YEAR)
    df_m = read_single_csv(f"JpLT_{region_nmb:02}-males.csv", YEAR)
    df_f = read_single_csv(f"JpLT_{region_nmb:02}-females.csv", YEAR)
    df_year = pd.concat([df_t, df_m, df_f], keys=("total", "males", "females")).T
    
    region_name = extract_name_of_region_from_file(region_nmb)
    df_year.index = [region_name]
    return df_year


df = pd.concat([read_single_region(nmb) for nmb in range(48)])
print(df.shape)
df.head()

(48, 15)


total                            males                           \
Age         0      65     80    100   110    0      65    80    100   110   
Japan     84.14  22.06  10.65  2.36  1.33  81.10  19.54  9.01  2.08  1.30   
Hokkaido  83.62  21.82  10.68  2.46  1.37  80.57  19.24  8.94  2.18  1.35   
Aomori    82.22  20.82   9.97  2.11  1.24  78.78  17.97  8.17  1.95  1.27   
Iwate     83.19  21.57  10.41  2.19  1.26  79.68  18.85  8.66  2.06  1.31   
Miyagi    84.10  22.01  10.58  2.35  1.33  81.08  19.55  8.99  2.02  1.27   

         females                            
Age          0      65     80    100   110  
Japan      87.16  24.40  11.85  2.40  1.33  
Hokkaido   86.47  24.08  11.92  2.51  1.37  
Aomori     85.55  23.37  11.15  2.13  1.24  
Iwate      86.77  24.10  11.62  2.21  1.26  
Miyagi     87.13  24.31  11.71  2.41  1.34

In [7]:
# rename columns
rename_dict = {
    ('total', 0)   : 't_0',
    ('total', 65)  : 't_65',
    ('total', 80)  : 't_80',
    ('total', 100) : 't_100',
    ('total', 110) : 't_110',
    ('males', 0)   : 'm_0',
    ('males', 65)  : 'm_65',
    ('males', 80)  : 'm_80',
    ('males', 100) : 'm_100',
    ('males', 110) : 'm_110',
    ('females', 0)   : 'f_0',
    ('females', 65)  : 'f_65',
    ('females', 80)  : 'f_80',
    ('females', 100) : 'f_100',
    ('females', 110) : 'f_110'
}

df.columns = [rename_dict[col] for col in df.columns]

# sort values
df = pd.concat([df.loc[['Japan']],
                df.drop(index='Japan').sort_values(by=SORTED_BY, ascending=False)])

print(df.shape)
df.head()

(48, 15)


,t_0,t_65,t_80,t_100,t_110,m_0,m_65,m_80,m_100,m_110,f_0,f_65,f_80,f_100,f_110
Japan,84.14,22.06,10.65,2.36,1.33,81.10,19.54,9.01,2.08,1.30,87.16,24.40,11.85,2.40,1.33
Nagano,85.28,22.90,11.03,2.25,1.27,82.58,20.59,9.44,2.02,1.26,87.96,25.07,12.26,2.30,1.27
Shiga,85.19,22.73,10.78,2.29,1.28,82.31,20.40,9.25,2.14,1.32,88.06,24.93,11.98,2.31,1.28
Nara,85.09,22.63,10.74,2.29,1.30,82.38,20.40,9.25,1.92,1.22,87.61,24.61,11.87,2.35,1.30
Kyoto,84.89,22.70,11.04,2.50,1.38,81.71,20.14,9.34,2.13,1.31,87.92,24.96,12.25,2.57,1.39


In [8]:
# calculate and insert "bonus" columns
df.insert(loc=1, column='bonus_0→65',    value=(df['t_65'] +65-df['t_0']  ).round(2))
df.insert(loc=3, column='bonus_65→80',   value=(df['t_80'] +15-df['t_65'] ).round(2))
df.insert(loc=5, column='bonus_80→100',  value=(df['t_100']+20-df['t_80'] ).round(2))
df.insert(loc=7, column='bonus_100→110', value=(df['t_110']+10-df['t_100']).round(2))

print(df.shape)
df.head()

(48, 19)


,t_0,bonus_0→65,t_65,bonus_65→80,t_80,bonus_80→100,t_100,bonus_100→110,t_110,m_0,m_65,m_80,m_100,m_110,f_0,f_65,f_80,f_100,f_110
Japan,84.14,2.92,22.06,3.59,10.65,11.71,2.36,8.97,1.33,81.10,19.54,9.01,2.08,1.30,87.16,24.40,11.85,2.40,1.33
Nagano,85.28,2.62,22.90,3.13,11.03,11.22,2.25,9.02,1.27,82.58,20.59,9.44,2.02,1.26,87.96,25.07,12.26,2.30,1.27
Shiga,85.19,2.54,22.73,3.05,10.78,11.51,2.29,8.99,1.28,82.31,20.40,9.25,2.14,1.32,88.06,24.93,11.98,2.31,1.28
Nara,85.09,2.54,22.63,3.11,10.74,11.55,2.29,9.01,1.30,82.38,20.40,9.25,1.92,1.22,87.61,24.61,11.87,2.35,1.30
Kyoto,84.89,2.81,22.70,3.34,11.04,11.46,2.50,8.88,1.38,81.71,20.14,9.34,2.13,1.31,87.92,24.96,12.25,2.57,1.39


In [9]:
df.loc[:, ['d_0']]   = (df['f_0']   - df['m_0']  ).round(2)
df.loc[:, ['d_65']]  = (df['f_65']  - df['m_65'] ).round(2)
df.loc[:, ['d_80']]  = (df['f_80']  - df['m_80'] ).round(2)
df.loc[:, ['d_100']] = (df['f_100'] - df['m_100']).round(2)
df.loc[:, ['d_110']] = (df['f_110'] - df['m_110']).round(2)

print(df.shape)
df.head()

(48, 24)


,t_0,bonus_0→65,t_65,bonus_65→80,t_80,bonus_80→100,t_100,bonus_100→110,t_110,m_0,...,f_0,f_65,f_80,f_100,f_110,d_0,d_65,d_80,d_100,d_110
Japan,84.14,2.92,22.06,3.59,10.65,11.71,2.36,8.97,1.33,81.10,...,87.16,24.40,11.85,2.40,1.33,6.06,4.86,2.84,0.32,0.03
Nagano,85.28,2.62,22.90,3.13,11.03,11.22,2.25,9.02,1.27,82.58,...,87.96,25.07,12.26,2.30,1.27,5.38,4.48,2.82,0.28,0.01
Shiga,85.19,2.54,22.73,3.05,10.78,11.51,2.29,8.99,1.28,82.31,...,88.06,24.93,11.98,2.31,1.28,5.75,4.53,2.73,0.17,-0.04
Nara,85.09,2.54,22.63,3.11,10.74,11.55,2.29,9.01,1.30,82.38,...,87.61,24.61,11.87,2.35,1.30,5.23,4.21,2.62,0.43,0.08
Kyoto,84.89,2.81,22.70,3.34,11.04,11.46,2.50,8.88,1.38,81.71,...,87.92,24.96,12.25,2.57,1.39,6.21,4.82,2.91,0.44,0.08


<br>

In [11]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df[['t_0', 't_65', 't_80', 't_100', 't_110',
                           'm_0', 'm_65', 'm_80', 'm_100', 'm_110',
                           'f_0', 'f_65', 'f_80', 'f_100', 'f_110',
                           'd_0', 'd_65', 'd_80', 'd_100', 'd_110'
                          ]],
                       row_center="Japan", nmb=5, max_lng=10)

Number of records: 48


,t_0,t_65,t_80,t_100,t_110,m_0,m_65,m_80,m_100,m_110,f_0,f_65,f_80,f_100,f_110,d_0,d_65,d_80,d_100,d_110
max,85.28 -Nagano,22.9 -Nagano,11.04 -Kyoto,2.55 -Tokyo,1.4 -Tokyo,82.58 -Nagano,20.59 -Nagano,9.44 -Nagano,2.24 -Tokyo,1.37 -Tokyo,88.14 -Shimane,25.11 -Shimane,12.31 -Shimane,2.6 -Tokyo,1.4 -Tokyo,7.13 -Okinawa,5.4 -Aomori,3.24 -Tottori,0.51 -Shimane,0.11 -Shimane
max_2,85.19 -Shiga,22.73 -Shiga,11.03 -Nagano,2.5 -Kyoto,1.38 -Kyoto,82.38 -Nara,20.4 -Shiga,9.38 -Okinawa,2.22 -Okinawa,1.36 -Okinawa,88.06 -Shiga,25.07 -Nagano,12.26 -Nagano,2.57 -Kyoto,1.39 -Kyoto,7.09 -Iwate,5.39 -Shimane,3.15 -Shimane,0.46 -Yamanashi,0.08 -Nara
max_3,85.09 -Nara,22.7 -Kyoto,11.03 -Shimane,2.47 -Kanagawa,1.38 -Osaka,82.31 -Shiga,20.4 -Nara,9.34 -Kyoto,2.18 -Kanagawa,1.35 -Osaka,87.96 -Nagano,24.96 -Kyoto,12.25 -Kyoto,2.52 -Kanagawa,1.38 -Osaka,6.9 -Kochi,5.39 -Tottori,3.06 -Yamanashi,0.46 -Tottori,0.08 -Kyoto
max_4,84.89 -Kyoto,22.63 -Nara,11.02 -Tokyo,2.47 -Osaka,1.37 -Kanagawa,81.98 -Fukui,20.14 -Kyoto,9.3 -Kumamoto,2.18 -Hokkaido,1.35 -Hokkaido,87.92 -Kyoto,24.93 -Shiga,12.25 -Tokyo,2.52 -Osaka,1.37 -Kanagawa,6.82 -Shimane,5.34 -Okinawa,3.05 -Ishikawa,0.45 -Ishikawa,0.08 -Ishikawa
max_5,84.85 -Fukui,22.52 -Shimane,11.0 -Okinawa,2.46 -Hokkaido,1.37 -Hokkaido,81.71 -Kyoto,19.97 -Kumamoto,9.26 -Tokyo,2.17 -Osaka,1.34 -Kanagawa,87.71 -Kumamoto,24.79 -Tokyo,12.21 -Okinawa,2.51 -Hokkaido,1.37 -Hokkaido,6.77 -Aomori,5.27 -Osaka,3.0 -Ehime,0.44 -Kyoto,0.08 -Yamanashi
Japan,– 84.14 –,– 22.06 –,– 10.65 –,– 2.36 –,– 1.33 –,– 81.1 –,– 19.54 –,– 9.01 –,– 2.08 –,– 1.3 –,– 87.16 –,– 24.4 –,– 11.85 –,– 2.4 –,– 1.33 –,– 6.06 –,– 4.86 –,– 2.84 –,– 0.32 –,– 0.03 –
min_5,83.34 -Ibaraki,21.6 -Fukushima,10.3 -Fukushima,2.21 -Gumma,1.27 -Nagano,80.22 -Miyazaki,19.06 -Osaka,8.65 -Fukushima,1.92 -Nara,1.23 -Akita,86.45 -Akita,23.94 -Akita,11.47 -Gumma,2.25 -Gumma,1.26 -Iwate,5.69 -Shizuoka,4.53 -Shiga,2.64 -Chiba,0.18 -Aomori,-0.04 -Nagasaki
min_4,83.28 -Fukushima,21.57 -Iwate,10.26 -Wakayama,2.2 -Yamagata,1.26 -Iwate,79.94 -Okinawa,19.04 -Wakayama,8.65 -Gumma,1.91 -Wakayama,1.22 -Tochigi,86.4 -Ibaraki,23.92 -Fukushima,11.46 -Akita,2.22 -Yamagata,1.26 -Kochi,5.69 -Fukui,4.51 -Yamagata,2.62 -Nara,0.17 -Shiga,-0.04 -Yamagata
min_3,83.23 -Akita,21.57 -Akita,10.25 -Tochigi,2.19 -Iwate,1.26 -Yamagata,79.92 -Akita,18.9 -Akita,8.61 -Wakayama,1.9 -Akita,1.22 -Gifu,86.38 -Tochigi,23.92 -Tochigi,11.45 -Wakayama,2.21 -Iwate,1.26 -Yamagata,5.49 -Gifu,4.48 -Nagano,2.61 -Kagawa,0.16 -Kochi,-0.04 -Shiga
min_2,83.19 -Iwate,21.56 -Gumma,10.23 -Gumma,2.15 -Akita,1.25 -Akita,79.68 -Iwate,18.85 -Iwate,8.59 -Akita,1.89 -Gifu,1.22 -Shimane,86.22 -Fukushima,23.91 -Gumma,11.45 -Mie,2.19 -Akita,1.25 -Akita,5.38 -Nagano,4.47 -Mie,2.58 -Mie,0.15 -Iwate,-0.05 -Iwate


In [12]:
mal.min_and_max_values(df[['t_0', 'bonus_0→65', 't_65', 'bonus_65→80', 't_80', 'bonus_80→100', 't_100', 'bonus_100→110', 't_110']],
                       row_center="Japan", nmb=5, max_lng=10)

Number of records: 48


,t_0,bonus_0→65,t_65,bonus_65→80,t_80,bonus_80→100,t_100,bonus_100→110,t_110
max,85.28 -Nagano,3.62 -Okinawa,22.9 -Nagano,4.15 -Aomori,11.04 -Kyoto,12.14 -Aomori,2.55 -Tokyo,9.13 -Aomori,1.4 -Tokyo
max_2,85.19 -Shiga,3.6 -Aomori,22.73 -Shiga,3.99 -Okinawa,11.03 -Nagano,12.04 -Wakayama,2.5 -Kyoto,9.1 -Akita,1.38 -Kyoto
max_3,85.09 -Nara,3.38 -Iwate,22.7 -Kyoto,3.86 -Hokkaido,11.03 -Shimane,11.98 -Gumma,2.47 -Kanagawa,9.07 -Gumma,1.38 -Osaka
max_4,84.89 -Kyoto,3.34 -Akita,22.63 -Nara,3.84 -Osaka,11.02 -Tokyo,11.98 -Tochigi,2.47 -Osaka,9.07 -Iwate,1.37 -Kanagawa
max_5,84.85 -Fukui,3.32 -Fukushima,22.52 -Shimane,3.84 -Iwate,11.0 -Okinawa,11.94 -Fukushima,2.46 -Hokkaido,9.06 -Yamagata,1.37 -Hokkaido
Japan,– 84.14 –,– 2.92 –,– 22.06 –,– 3.59 –,– 10.65 –,– 11.71 –,– 2.36 –,– 8.97 –,– 1.33 –
min_5,83.34 -Ibaraki,2.67 -Ishikawa,21.6 -Fukushima,3.29 -Gifu,10.3 -Fukushima,11.46 -Kyoto,2.21 -Gumma,8.91 -Hokkaido,1.27 -Nagano
min_4,83.28 -Fukushima,2.62 -Nagano,21.57 -Iwate,3.21 -Fukui,10.26 -Wakayama,11.44 -Okinawa,2.2 -Yamagata,8.91 -Osaka,1.26 -Iwate
min_3,83.23 -Akita,2.54 -Nara,21.57 -Akita,3.13 -Nagano,10.25 -Tochigi,11.36 -Kumamoto,2.19 -Iwate,8.9 -Kanagawa,1.26 -Yamagata
min_2,83.19 -Iwate,2.54 -Shiga,21.56 -Gumma,3.11 -Nara,10.23 -Gumma,11.33 -Shimane,2.15 -Akita,8.88 -Kyoto,1.25 -Akita


<br>

In [14]:
dd_replacement = {
    'Japan'     : {'jp': ('日本', ''), 'en': ('Japan', ''), 'ru': ('Япония', '')},
    'Hokkaido'  : {'jp': ('北海道', ''), 'en': ('Hokkaido', 'Hokkaido'), 'ru': ('Хокка́йдо', 'Хоккайдо')},
    'Aomori'    : {'jp': ('青森県', ''), 'en': ('Aomori ', 'Aomori Prefecture'), 'ru': ('Ао́мори', 'Аомори (префектура)')},
    'Iwate'     : {'jp': ('岩手県', ''), 'en': ('Iwate', 'Iwate Prefecture'), 'ru': ('Ива́те', 'Ивате (префектура)')},
    'Miyagi'    : {'jp': ('宮城県', ''), 'en': ('Miyagi', 'Miyagi Prefecture'), 'ru': ('Мия́ги', 'Мияги (префектура)')},
    'Akita'     : {'jp': ('秋田県', ''), 'en': ('Akita', 'Akita Prefecture'), 'ru': ('А́кита', 'Акита (префектура)')},
    'Yamagata'  : {'jp': ('山形県', ''), 'en': ('Yamagata', 'Yamagata Prefecture'), 'ru': ('Ямага́та', 'Ямагата (префектура)')},
    'Fukushima' : {'jp': ('福島県', ''), 'en': ('Fukushima', 'Fukushima Prefecture'), 'ru': ('Фуку́сима', 'Фукусима (префектура)')},
    'Ibaraki'   : {'jp': ('茨城県', ''), 'en': ('Ibaraki', 'Ibaraki Prefecture'), 'ru': ('Ибара́ки', 'Ибараки')},
    'Tochigi'   : {'jp': ('栃木県', ''), 'en': ('Tochigi', 'Tochigi Prefecture'), 'ru': ('Тоти́ги', 'Тотиги (префектура)')},
    'Gumma'     : {'jp': ('群馬県', ''), 'en': ('Gunma', 'Gunma Prefecture'), 'ru': ('Гумма', 'Гумма (префектура)')},
    'Saitama'   : {'jp': ('埼玉県', ''), 'en': ('Saitama', 'Saitama Prefecture'), 'ru': ('Са́йтама', 'Сайтама (префектура)')},
    'Chiba'     : {'jp': ('千葉県', ''), 'en': ('Chiba', 'Chiba Prefecture'), 'ru': ('Тиба', 'Тиба (префектура)')},
    'Tokyo'     : {'jp': ('東京都', ''), 'en': ('Tokyo', 'Tokyo'), 'ru': ('То́кио', 'Токио')},
    'Kanagawa'  : {'jp': ('神奈川県', ''), 'en': ('Kanagawa', 'Kanagawa Prefecture'), 'ru': ('Канага́ва', 'Канагава')},
    'Niigata'   : {'jp': ('新潟県', ''), 'en': ('Niigata', 'Niigata Prefecture'), 'ru': ('Ниига́та', 'Ниигата (префектура)')},
    'Toyama'    : {'jp': ('富山県', ''), 'en': ('Toyama', 'Toyama Prefecture'), 'ru': ('Тоя́ма', 'Тояма (префектура)')},
    'Ishikawa'  : {'jp': ('石川県', ''), 'en': ('Ishikawa', 'Ishikawa Prefecture'), 'ru': ('Исика́ва', 'Исикава')},
    'Fukui'     : {'jp': ('福井県', ''), 'en': ('Fukui', 'Fukui Prefecture'), 'ru': ('Фукуи́', 'Фукуи (префектура)')},
    'Yamanashi' : {'jp': ('山梨県', ''), 'en': ('Yamanashi', 'Yamanashi Prefecture'), 'ru': ('Ямана́си', 'Яманаси (префектура)')},
    'Nagano'    : {'jp': ('長野県', ''), 'en': ('Nagano', 'Nagano Prefecture'), 'ru': ('Нага́но', 'Нагано (префектура)')},
    'Gifu'      : {'jp': ('岐阜県', ''), 'en': ('Gifu', 'Gifu Prefecture'), 'ru': ('Ги́фу', 'Гифу (префектура)')},
    'Shizuoka'  : {'jp': ('静岡県', ''), 'en': ('Shizuoka', 'Shizuoka Prefecture'), 'ru': ('Сидзуо́ка', 'Сидзуока (префектура)')},
    'Aichi'     : {'jp': ('愛知県', ''), 'en': ('Aichi', 'Aichi Prefecture'), 'ru': ('А́йти', 'Айти (префектура)')},
    'Mie'       : {'jp': ('三重県', ''), 'en': ('Mie', 'Mie Prefecture'), 'ru': ('Ми́э', 'Миэ')},
    'Shiga'     : {'jp': ('滋賀県', ''), 'en': ('Shiga', 'Shiga Prefecture'), 'ru': ('Си́га', 'Сига')},
    'Kyoto'     : {'jp': ('都府', ''), 'en': ('Kyoto', 'Kyoto Prefecture'), 'ru': ('Кио́то', 'Киото (префектура)')},
    'Osaka'     : {'jp': ('大阪府', ''), 'en': ('Osaka', 'Osaka Prefecture'), 'ru': ('О́сака', 'Осака (префектура)')},
    'Hyogo'     : {'jp': ('兵庫県', ''), 'en': ('Hyōgo', 'Hyōgo Prefecture'), 'ru': ('Хио́го', 'Хиого')},
    'Nara'      : {'jp': ('奈良県', ''), 'en': ('Nara', 'Nara Prefecture'), 'ru': ('На́ра', 'Нара (префектура)')},
    'Wakayama'  : {'jp': ('和歌山県', ''), 'en': ('Wakayama', 'Wakayama Prefecture'), 'ru': ('Вакая́ма', 'Вакаяма (префектура)')},
    'Tottori'   : {'jp': ('鳥取県', ''), 'en': ('Tottori', 'Tottori Prefecture'), 'ru': ('Тотто́ри', 'Тоттори (префектура)')},
    'Shimane'   : {'jp': ('島根県', ''), 'en': ('Shimane', 'Shimane Prefecture'), 'ru': ('Сима́не', 'Симане')},
    'Okayama'   : {'jp': ('岡山県', ''), 'en': ('Okayama', 'Okayama Prefecture'), 'ru': ('Окаяма', 'Окаяма (префектура)')},
    'Hiroshima' : {'jp': ('広島県', ''), 'en': ('Hiroshima', 'Hiroshima Prefecture'), 'ru': ('Хироси́ма', 'Хиросима (префектура)')},
    'Yamaguchi' : {'jp': ('山口県', ''), 'en': ('Yamaguchi', 'Yamaguchi Prefecture'), 'ru': ('Ямагу́ти', 'Ямагути (префектура)')},
    'Tokushima' : {'jp': ('徳島県', ''), 'en': ('Tokushima', 'Tokushima Prefecture'), 'ru': ('Токуси́ма', 'Токусима (префектура)')},
    'Kagawa'    : {'jp': ('香川県', ''), 'en': ('Kagawa', 'Kagawa Prefecture'), 'ru': ('Кагава', 'Кагава (префектура)')},
    'Ehime'     : {'jp': ('愛媛県', ''), 'en': ('Ehime', 'Ehime Prefecture'), 'ru': ('Эхиме́', 'Эхиме')},
    'Kochi'     : {'jp': ('高知県', ''), 'en': ('Kōchi', 'Kōchi Prefecture'), 'ru': ('Ко́ти', 'Коти (префектура)')},
    'Fukuoka'   : {'jp': ('福岡県', ''), 'en': ('Fukuoka', 'Fukuoka Prefecture'), 'ru': ('Фукуо́ка', 'Фукуока (префектура)')},
    'Saga'      : {'jp': ('佐賀県', ''), 'en': ('Saga', 'Saga Prefecture'), 'ru': ('Са́га', 'Сага (префектура)')},
    'Nagasaki'  : {'jp': ('長崎県', ''), 'en': ('Nagasaki', 'Nagasaki Prefecture'), 'ru': ('Нагаса́ки', 'Нагасаки (префектура)')},
    'Kumamoto'  : {'jp': ('熊本県', ''), 'en': ('Kumamoto', 'Kumamoto Prefecture'), 'ru': ('Кумамо́то', 'Кумамото (префектура)')},
    'Oita'      : {'jp': ('大分県', ''), 'en': ('Ōita', 'Ōita Prefecture'), 'ru': ('О́ита', 'Оита (префектура)')},
    'Miyazaki'  : {'jp': ('宮崎県', ''), 'en': ('Miyazaki', 'Miyazaki Prefecture'), 'ru': ('Миядза́ки', 'Миядзаки (префектура)')},
    'Kagoshima' : {'jp': ('鹿児島県', ''), 'en': ('Kagoshima', 'Kagoshima Prefecture'), 'ru': ('Каго́сима', 'Кагосима (префектура)')},
    'Okinawa'   : {'jp': ('沖縄県', ''), 'en': ('Okinawa', 'Okinawa Prefecture'), 'ru': ('Окина́ва', 'Окинава (префектура)')}
}

In [15]:
# create code for Wikipedia table
def create_table_code_v1(df, lang='en'):

    def if_value(x, prec=2):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=2):  # 'change_value'
        # decoration of colored values, that shows change of regular values
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'color:darkgray;"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=2):
        # decoration of colored values of bold font
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    # Load code for header of the table from file. Depending on condions (see above), code is loaded from various files.
    with open(f"design/Japan_header_year_detailed_v1_{lang}.txt", mode='r', encoding="utf-8") as fh:
        st_header = fh.read().strip()
    
    # Create table code record by record
    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        
        # records in bold
        if ser.name == "Japan":
            st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_0→65"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_65→80"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_80→100"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_100"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["bonus_100→110"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_110"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| \'\'\'{if_value(ser["m_0"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_65"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_80"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_100"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_110"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;border-left-width:2px;"| \'\'\'{if_value(ser["f_0"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_65"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_80"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_100"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_110"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["d_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_100"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_110"])}\'\'\''
        else:
            name_link = dd_replacement[ser.name][lang][1]
            name_visible = dd_replacement[ser.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_0"])} ' + \
                  f'|| {if_value(ser["bonus_0→65"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_65"])} ' + \
                  f'|| {if_value(ser["bonus_65→80"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_80"])} ' + \
                  f'|| {if_value(ser["bonus_80→100"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_100"])} ' + \
                  f'|| {if_value(ser["bonus_100→110"])} ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_110"])} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| {if_value(ser["m_0"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_65"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_80"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_100"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_110"])} ' + \
                  f'||style="background:#fee7f6;border-left-width:2px;"| {if_value(ser["f_0"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_65"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_80"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_100"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_110"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["d_0"])} ' + \
                  f'|| {if_value(ser["d_65"])} ' + \
                  f'|| {if_value(ser["d_80"])} ' + \
                  f'|| {if_value(ser["d_100"])} ' + \
                  f'|| {if_value(ser["d_110"])}'

    st += '\n|}'
    
    # fix microbugs that can happened due to nuances of calculations and rounding
    st = st.replace('color:crimson;"| −0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:darkgreen;"| 0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:crimson;"| \'\'\'−0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')  \
           .replace('color:darkgreen;"| \'\'\'0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')
      
    # since in Russian standard decimal numbers use a comma, made in numbers replacement of dot to comma
    # WORKING WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    # set gray color of text for missing values that in the table are represented with '—'
    st = st.replace(';"| —', ';color:silver;"| —')
    
    # join code of header of the table with code for records
    st = st_header + st
    
    return st

table_code = create_table_code_v1(df, lang='en')
with open(f"output/Table code for Japanese regions in {YEAR} -v1 -en.txt", 'w', encoding="utf-8") as fh:
    fh.write(table_code)
    
# table_code = create_table_code_v1(df, lang='ru')
# with open(f"output/Table code for Japanese regions in {YEAR} -v1 -ru.txt", 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

In [16]:
# create code for Wikipedia table
def create_table_code_v2(df, lang='en'):

    def if_value(x, prec=2):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=2):  # 'change_value'
        # decoration of colored values, that shows change of regular values
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'color:darkgray;"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=2):
        # decoration of colored values of bold font
        return '"| —' if math.isnan(x) else \
               f'color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    # Load code for header of the table from file. Depending on condions (see above), code is loaded from various files.
    with open(f"design/Japan_header_year_detailed_v2_{lang}.txt", mode='r', encoding="utf-8") as fh:
        st_header = fh.read().strip()
    
    # Create table code record by record
    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        
        # records in bold
        if ser.name == "Japan":
            st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#e0ffd8;"| \'\'\'{if_value(ser["t_0"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_0"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_0"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_0"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_0→65"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_65"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_65"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_65"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_65"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_65→80"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_80"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_80"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_80"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_80"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_80→100"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_100"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_100"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_100"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_100"])}\'\'\' ' + \
                  f'||style="border-left-width:2px;"| \'\'\'{if_value(ser["bonus_100→110"])}\'\'\' ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| \'\'\'{if_value(ser["t_110"])}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["m_110"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["f_110"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["d_110"])}\'\'\''
            
        else:
            name_link = dd_replacement[ser.name][lang][1]
            name_visible = dd_replacement[ser.name][lang][0]
            name_inserted = name_link if name_link == name_visible else f"{name_link}|{name_visible}"
            st += '\n' + '|-\n' + \
                  f'| [[{name_inserted}]] ' + \
                  f'||style="background:#e0ffd8;"| {if_value(ser["t_0"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_0"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_0"])} ' + \
                  f'|| {if_value(ser["d_0"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_0→65"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_65"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_65"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_65"])} ' + \
                  f'|| {if_value(ser["d_65"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_65→80"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_80"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_80"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_80"])} ' + \
                  f'|| {if_value(ser["d_80"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_80→100"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_100"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_100"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_100"])} ' + \
                  f'|| {if_value(ser["d_100"])} ' + \
                  f'||style="border-left-width:2px;"| {if_value(ser["bonus_100→110"])} ' + \
                  f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["t_110"])} ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["m_110"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["f_110"])} ' + \
                  f'|| {if_value(ser["d_110"])}'

    st += '\n|}'
    
    # fix microbugs that can happened due to nuances of calculations and rounding
    st = st.replace('color:crimson;"| −0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:darkgreen;"| 0.00 ', 'color:darkgray;"| 0.00 ')  \
           .replace('color:crimson;"| \'\'\'−0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')  \
           .replace('color:darkgreen;"| \'\'\'0.00\'\'\' ', 'color:darkgray;"| \'\'\'0.00\'\'\' ')
    
      
    # since in Russian standard decimal numbers use a comma, made in numbers replacement of dot to comma
    # WORKING WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    # set gray color of text for missing values that in the table are represented with '—'
    st = st.replace(';"| —', ';color:silver;"| —')
    
    # join code of header of the table with code for records
    st = st_header + st
    
    return st

table_code = create_table_code_v2(df, lang='ru')
with open(f"output/Table code for Japanese regions in {YEAR} -v2 -ru.txt", 'w', encoding="utf-8") as fh:
    fh.write(table_code)

table_code = create_table_code_v2(df, lang='en')
with open(f"output/Table code for Japanese regions in {YEAR} -v2 -en.txt", 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br>

In [18]:
# play beep to denote completion of the program
import IPython.display as ipd
import numpy as np

# manually generated sound
t = 1  # time is seconds
beep = np.sin(2*np.pi*400*np.arange(10000*t)/10000)
ipd.Audio(beep, rate=10000, autoplay=True)